# Modèle linéaire

Considérons la cas classique d'une fonction affine :

$$y=ax+b$$

Ici, $a$ et $b$ sont des réels. Ces deux nombres définissent entièrement la courbe et permet donc d'obtenir une relation **affine** entre $x$ et $y$. En statistique, cette relation est à la base des modèles dit **linéaires**, où une variable réponse se définit comme une somme de variables explicatives où chacune de ces dernières sont multipliés par un coefficient.


## Modèle linéaire simple

![](https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Linear_regression.svg/438px-Linear_regression.svg.png)

Dans le modèle linéaire simple (une seule variable explicative), on suppose que la variable réponse suit le modèle suivant :

$$y_i=\beta_0 + \beta_1 x_i + \varepsilon_i$$

On remarque la ressemblance avec la fonction affine présentée ci-dessus. La différence réside dans l'existence du terme aléatoire (appelé bruit) $\varepsilon_i$. Afin de considérer le modèle, il est nécessaire de se placer sous les hypothèses suivantes

$$(\mathcal{H}) : \left\{\begin{matrix}
\mathbb{E}[\varepsilon_i]=0\\ 
\text{Cov}(\varepsilon_i, \varepsilon_j)=\delta_{ij} \sigma^2
\end{matrix}\right.$$
Les différents éléments qui interviennent sont :

- $\beta_0$ : l'ordonnée à l'origine (nommée *intercept*)
- $\beta_1$ : le coefficient directeur
- $x_i$ : l'observation $i$
- $y_i$ : le $i$-ème prix
- $\varepsilon_i$ : le bruit aléatoire liée à la $i$-ème observation

La solution peut se calculer facilement via les formules fermées suivantes :

$$\hat{\beta}_1=\frac{\sum_{i=1}^n (x_i - \bar{x})(y_i - \bar{y})}{\sum_{i=1}^n (x_i - \bar{x})^2} \qquad \hat{\beta}_0 = \hat{y} - \hat{\beta}_1 \bar{x}$$

## Modèle linéaire multiple

Dans le cas multiple (pour $p$ variables explicatives), pour la $i$-ème observation, le modèle s'écrit :

$$y_i= \beta_0 + \sum_{j=1}^p \beta_j x_{ij} + \varepsilon_i$$

Ainsi, une observation $x_i$ n'est plus une valeur, mais un **vecteur** $(x_{i1}, \dots, x_{ip})$. Il est plus commode de regrouper ces prix $y_i$ et ces vecteurs d'observations $x_i$ dans des matrices :

$$Y=X \beta + \varepsilon$$

Sous les hypothèses équivalentes du modèle simple en plus grand dimension

$$(\mathcal{H}) : \left\{\begin{matrix}
\text{rank}(X)=p\\ 
\mathbb{E}[\varepsilon]=0 \text{ et }\text{Var}(\varepsilon)=\sigma^2 I_p
\end{matrix}\right.$$

Les différents éléments qui interviennent sont :

- $\beta$ : le vecteur directeur
- $X$ : la matrice des observations
- $Y$ : le vecteur de prix
- $\varepsilon$ : le vecteur de bruit

Avec $X=( \mathbf{1}, X_1, \dots, X_n)$, $Y=(y_1, \dots, y_n)^\top$ et $\varepsilon=(\varepsilon_1, \dots, \varepsilon_n)^\top$. La solution des MCO (Moindres Carrés Ordinaires) est alors :

$$\hat{\beta}= (X^\top X)^{-1} X^\top Y$$

Vous pouvez d'ailleurs faire la démonstration de votre coté ! Pour plus d'information mathématiques, je vous conseil le portail de wikipédia qui est très bien fait : [lien ici](https://fr.wikipedia.org/wiki/Portail:Probabilit%C3%A9s_et_statistiques)

In [63]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Implémenter une régression linéaire 


In [0]:
#importer vos librairies 
import pandas as pd 
import numpy as np 
import io
from sklearn import linear_model 
from sklearn.metrics import mean_squared_error, r2_score

In [65]:
#charger les données 
#price_availability.csv
#listings_final.csv
#attention aux valeurs manquantes !!
root_path = '/content/drive/My Drive/Colab Notebooks'
price = pd.read_csv(root_path+"/data/price_availability.csv", sep=';')
listing = pd.read_csv(root_path+"/data/listings_final.csv", sep=';')
listing = listing.drop(589)
print('data loaded')

data loaded


In [66]:
price.head()

,listing_id,day,created,available,local_currency,local_price,min_nights
0,9810829,2018-12-08,2018-09-27 06:14:10.000+0000,True,EUR,160,1
1,9810829,2018-12-08,2018-09-26 19:34:02.000+0000,True,EUR,160,1
2,20897010,2018-12-09,2018-09-27 10:38:57.000+0000,True,EUR,172,2
3,20897010,2018-12-09,2018-09-27 06:10:27.000+0000,True,EUR,172,2
4,20897010,2018-12-09,2018-09-26 19:30:25.000+0000,True,EUR,172,2


In [67]:
price.describe()

,listing_id,local_price,min_nights
count,4.748696e+06,4.748696e+06,4.748696e+06
mean,1.523313e+07,1.944215e+02,7.877554e+00
std,9.190721e+06,2.289745e+02,2.407054e+01
min,5.396000e+03,9.000000e+00,1.000000e+00
25%,6.351188e+06,7.900000e+01,2.000000e+00
50%,1.641294e+07,1.440000e+02,3.000000e+00
75%,2.353692e+07,2.400000e+02,4.000000e+00
max,2.885198e+07,1.000300e+04,3.650000e+02


In [68]:
listing.head()

,Unnamed: 0,listing_id,name,type,city,neighborhood,latitude,longitude,person_capacity,beds,bedrooms,bathrooms,is_rebookable,is_new_listing,is_fully_refundable,is_host_highly_rated,is_business_travel_ready,pricing_weekly_factor,pricing_monthly_factor
0,0,28581061,La maison Clery,private_room,Paris,2e arrondissement,48.869292,2.348335,1,1,1,2.0,False,True,True,False,False,1.00,1.00
1,1,661961,studio PARIS PLACE EDITH PIAF 75020,entire_home,Paris,NaN,48.867284,2.403255,2,1,1,1.0,False,False,True,True,False,0.88,0.69
2,2,1261705,chambre privée à louer @ paris oberkampf,private_room,Paris,NaN,48.867894,2.375897,1,1,1,1.0,False,False,True,True,False,1.00,1.00
3,3,1318834,Appartement au coeur du Marais,entire_home,Paris,République,48.870370,2.358510,3,2,2,1.0,False,False,True,False,False,0.82,0.48
4,4,1677091,Lovely & Quiet flat,entire_home,Paris,Buttes-Chaumont - Belleville,48.874149,2.373700,2,1,1,1.0,False,False,True,True,False,0.95,0.90


In [69]:
listing.describe()

,Unnamed: 0,listing_id,latitude,longitude,person_capacity,beds,bedrooms,bathrooms,pricing_weekly_factor,pricing_monthly_factor
count,999.000000,9.990000e+02,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000
mean,499.410410,1.543276e+07,48.864251,2.345202,3.669670,2.058058,1.326326,1.230230,0.942882,0.871809
std,288.950197,9.305853e+06,0.009824,0.031837,2.190568,1.500378,0.998806,0.524497,0.073776,0.162974
min,0.000000,5.609300e+04,48.844372,2.268992,1.000000,0.000000,0.000000,0.000000,0.600000,0.330000
25%,249.500000,6.341818e+06,48.856929,2.326608,2.000000,1.000000,1.000000,1.000000,0.900000,0.750000
50%,499.000000,1.727835e+07,48.864368,2.349152,3.000000,2.000000,1.000000,1.000000,1.000000,1.000000
75%,749.500000,2.378911e+07,48.871906,2.366973,4.000000,3.000000,2.000000,1.000000,1.000000,1.000000
max,999.000000,2.879280e+07,48.882691,2.412700,16.000000,16.000000,6.000000,5.000000,1.000000,1.000000


## Données d'entrée

L'objectif ici est de charger les données pour créer les matrices $X$ et $Y$ du modèle linéaire. **Attention**, il n'est pas nécessaire de rajouter le vecteur colonne $\mathbf{1}$ en première colonne, car *scikit-learn* le fait automatiquement !

In [0]:
#définir vos variables de travail X et Y
list(listing.columns)
X = listing.loc[:, ['listing_id', 'person_capacity', 'bedrooms', 'bathrooms']]
Y = []

In [0]:
#construire l'array des prix 
 
for i, row in X.iterrows():
    y = 0 
    ID = int(row['listing_id'])
    subset = price[price['listing_id'] == ID]
    y = subset['local_price'].mean()
    Y.append(y)

In [0]:
Y = np.asarray(Y)

En *Machine Learning*, on a l'habitude de couper l'ensemble de données en deux sous-ensembles :

- Un ensemble d'entraînement (*train set*), sur lequel le modèle va être calibré.
- Un ensemble de test (*test set*), qui ne sera pas utilisé pendant le calibrage mais permettra de vérifier l'aptitude du modèle à généraliser sur de nouvelles observations inconnues.

En général, on découpe l'ensemble de données (*split*) en prenant $\alpha \%$ de l'ensemble pour entraînement et $1-\alpha \%$ comme test. Dans la plus part des cas, on considère que $\alpha=10,20\ ou \ 30\%$.

In [0]:
#utiliser la méthode split de sklearn en splitant avec un alpha=30 et un random state=42 
#afficher la shape de vos données 
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.3, random_state=42)

In [75]:
X.shape

(999, 4)

In [76]:
Y.shape

(999,)

In [77]:
X_train.shape

(699, 4)

In [78]:
X_test.shape

(300, 4)

In [79]:
y_train.shape

(699,)

In [80]:
y_test.shape

(300,)

## Entraînement

Pour information, *scikit-learn* utilise le solveur OLS (Ordinary Least Squares) de *numpy*.

In [0]:
#créer l'objet de régression et entrainer le sur notre ensemble d'entraînement
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)

On affiche le vecteur des coefficients pour interpréter rapidement le modèle.

In [81]:
#afficher les coefficients
#que remarquez vous ? 
print('Coefficients beta_j: \n', regr.coef_)
print('Coefficients intercept beta_0: \n', regr.intercept_)

Coefficients beta_j: 
 [2.47834609e-06 3.23451085e+01 1.43887639e+01 7.75318453e+01]
Coefficients intercept beta_0: 
 -79.79953133672873


## Validation du modèle

### Le coefficient de détermination $R^2$

Par la suite, nous ferons l'hypothèse de gaussianité sur les bruits. Dans l'idée, nous aimerions obtenir une valeur numérique qui nous indique à quel point la régression linéaire a un sens sur nos données. Pour cela, introduisons les notations suivantes :

- $SCT=\|Y-\hat{y} \mathbf{1}\|^2$ est la somme des carrés totaux
- $SCE=\|\hat{Y}-\hat{y} \mathbf{1}\|^2$ est la somme des carrés expliqués
- $SCR=\|\hat{\varepsilon}\|^2$ est la somme des carrés résiduels

L'idée est de décomposer la somme des carrés totaux comme la somme des carrés que le modèle explique, en plus de la somme des carrés qui sont liés aux résidus (et donc que le modèle ne peut pas expliquer). On voit donc ici l'intérêt de calculer un coefficient à partir du $SCE$. Puisque l'on a la relation suivante :

$$SCT=SCE+SCR \text{ alors } 1=\frac{SCE}{SCT}+\frac{SCR}{SCT}$$

Plus les résidus sont petits (et donc la régression est "bonne"), plus $SCR$ devient petit et donc $SCE$ devient grand. Le schéma inverse s'opère de la même façon. Dans le meilleur des cas, on obtient $SCR=0$ et donc $SCE=SCT$ d'où le premier membre vaut $1$. Dans le cas contraite, $SCE=0$ et automatiquement, le premier membre est nul. C'est ainsi que l'on définit le coefficient de détermination $R^2$ comme 
$$R^2=\frac{SCE}{SCT}=1-\frac{SCR}{SCT}$$
Ainsi, $R^2 \in [0,1]$. Plus $R^2$ est proche de $1$, plus la régression linéaire a du sens. Au contraire, si $R^2$ est proche de $0$, le modèle linéaire possède un faible pouvoir explicatif.

In [87]:
#faire une prediction sur X_test
y_pred = regr.predict(X_test)
print("Taille de Y_pred : ", y_pred.size)

Taille de Y_pred :  300


In [98]:
#afficher l'erreur des moindres carrées sur l'ensemble d'entrainement ainsi que le R2
#que remarquez vous ?
print("Mean squared error : %.2f" % mean_squared_error(y_test, y_pred))
print("R2 : %.2f" % r2_score(y_test, y_pred))

Mean squared error : 19631.83
R2 : 0.42


In [0]:
# 